In [3]:
import sys
sys.path.append("/usr/local/Cellar/python\@3.8/3.8.3/bin/python3.8")
import ROOT
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import root_numpy
from root_pandas import read_root
from scipy import stats
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
#/usr/local/Cellar/jupyterlab/2.1.4/libexec/bin/python3.8 -m pip install

Welcome to JupyROOT 6.20/04


In [4]:
rootPath = '/Users/seanfoster/gitHub/g-2/TrackingAnalysis/TrackerTarget/trackerTarget.root'
#treeName = 'TrackerTargetTreeNominal'
treeName = 'TrackerTargetTreeTrueExtrap'

In [5]:
df = read_root(rootPath,treeName)
print(len(df))
print(df)

79946
           runNum  subRunNum  eventNum  islandNum  pdgID  \
0      1595008785          1         3          0    -13   
1      1595008785          1         4          0    -13   
2      1595008785          1        10          0    -13   
3      1595008785          1        11          0    -13   
4      1595008785          1        19          0    -13   
...           ...        ...       ...        ...    ...   
79941  1595184572         94      9962          0    -13   
79942  1595184572         94      9968          0    -13   
79943  1595184572         94      9982          0    -13   
79944  1595184572         94      9986          0    -13   
79945  1595184572         94      9990          0    -13   

                                              planeTimes  \
0      [0.434981832562994, 0.4556628064652798, 0.4763...   
1      [0.4205032022216324, 0.4411841761239182, 0.462...   
2      [0.4201023828104627, 0.4407833567127485, 0.461...   
3      [0.4214070813620434, 0.442

In [44]:
print(np.sqrt(df['planeMomXs'][0]*df['planeMomXs'][0]+df['planeMomYs'][0]*df['planeMomYs'][0]+df['planeMomZs'][0]*df['planeMomZs'][0]))
print(np.sqrt(df['ghostPlaneMomXs'][0]*df['ghostPlaneMomXs'][0]+df['ghostPlaneMomYs'][0]*df['ghostPlaneMomYs'][0]+df['ghostPlaneMomZs'][0]*df['ghostPlaneMomZs'][0]))
#print(df['planeMomXs'][4])
#print(df['ghostPlaneMomXs'][4])

[2920.40559469 2920.40559469 2920.40559469 2920.40559469 2920.40559469
 2920.40559469 2920.40559469 2920.40559469 2920.40559469 2920.40559469
 2920.40559469 2920.40559469 2920.40559469 2920.40559469 2920.40559469
 2920.40559469 2920.40559469 2920.40559469 2920.40559469 2920.40559469
 2920.40559469 2920.40559469 2920.40559469 2920.40559469 2920.40559469
 2920.40559469 2920.40559469 2920.40559469 2920.40559469 2920.40559469
 2920.40559469]
[3094.34963118 3094.34963118 3094.34963118 3094.34963118 3094.34963118
 3094.34963118 3094.34963118 3094.34963118 3093.64926288 3067.41614831
 3051.01193789 3030.63989748 3016.59931669 3000.36979307 2986.77801266
 2973.633022   2972.98543291 2972.98543291 2972.98543291 2972.98543291
 2972.98543291 2972.98543291 2972.98543291 2972.98543291 2972.98543291
 2972.98543291 2972.98543291 2972.98543291 2972.98543291 2972.98543291
 2972.98543291]


In [68]:
runNumDigits = len(str(df['runNum'].values.max()))
subRunNumDigits = len(str(df['subRunNum'].values.max()))
eventNumDigits = len(str(df['eventNum'].values.max()))
islandNumDigits = len(str(df['islandNum'].values.max()))

df['uniqueID'] = df.apply(
    lambda row: row['runNum']+row['subRunNum']*np.power(10,runNumDigits-1) + row['eventNum']*np.power(10,runNumDigits+subRunNumDigits-1),axis=1)

In [9]:
df["TrackNumber"] = range(0,len(df))
df0 = df[['TrackNumber','ghostPlaneAzimuths','ghostPlanePosXs','ghostPlanePosYs','ghostPlanePosZs']].set_index(['TrackNumber']).apply(pd.Series.explode).reset_index()#.set_index(['TrackNumber','planeAzimuths'])

KeyboardInterrupt: 

In [23]:
df0

,TrackNumber,ghostPlaneAzimuths,ghostPlanePosXs,ghostPlanePosYs,ghostPlanePosZs,planeRadPos,ghostPlaneRadPos
0,0,159.5,-6661.84,-0.298641,2490.76,0,NaN
1,0,159.55,-6664.01,-0.298641,2484.94,0,NaN
2,0,159.6,-6666.18,-0.298641,2479.13,0,NaN
3,0,159.65,-6668.34,-0.298641,2473.31,0,NaN
4,0,159.7,-6670.49,-0.298641,2467.49,0,NaN
...,...,...,...,...,...,...,...
2441876,79945,160.8,-6714.39,-0.797684,2338.2,79945,NaN
2441877,79945,160.85,-6716.29,-0.871276,2332.29,79945,NaN
2441878,79945,160.9,-6718.18,-0.944867,2326.38,79945,NaN
2441879,79945,160.95,-6720.06,-1.01846,2320.47,79945,NaN


In [102]:
iterables = [range(0,maxNumberOfTracks),
          azimuths]
index = pd.MultiIndex.from_product(iterables, names=['TrackNumber', 'Azimuth'])
df = pd.DataFrame(index=index)

In [115]:
values = []
name = "pdgID"

thisTrackNumber = 0
for entry in tree :
    if (thisTrackNumber < maxNumberOfTracks) :
        for azimuth in azimuths :
            values.append(entry.pdgID)
            
            ind = np.where(np.asarray(entry.planeAzimuths)==azimuth)[0]
            if (len(ind) > 0) :
                ind = int(ind[0])
            print(entry.planeAzimuths[ind])

    thisTrackNumber += 1
    
df['pdgID'] = values

160.0


TypeError: object of type 'int' has no len()

In [104]:
df

pdgID
TrackNumber Azimuth       
0           160.00     -13
            160.05     -13
            160.10     -13
            160.15     -13
            160.20     -13
...                    ...
9           160.50     -13
            160.55     -13
            160.60     -13
            160.65     -13
            160.70     -13

[150 rows x 1 columns]